In [1]:
import numpy as np
import argparse
import math
import os
import json
from nuscenes.nuscenes import NuScenes

base_dir = f'/disk/vanishing_data/ju878/nusc_kitti_test/train/oxts/'
dataroot = f'/disk/ml/datasets/nuScenes/'
TIMESTAMPS_FILENAME = "timestamps.txt"
OXTS_LINE_LEN = 6

json_tokens = f'/disk/ml/own_datasets/CODA/nuscenes_indices.json'

with open(json_tokens, 'r') as f:
    nuscenes_tokens = json.load(f)


In [2]:
nusc = NuScenes(dataroot=dataroot, version='v1.0-trainval', verbose=False)

In [20]:
def load_oxts_data(scene_token: str, base_dir):
    
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    
    number_of_samples = 1
    while sd_rec['next'] != '':
        number_of_samples += 1
        sd_rec = nusc.get('sample_data', sd_rec['next'])
        
    oxts_data = np.zeros((number_of_samples, OXTS_LINE_LEN))
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])

    for i in range(number_of_samples):
        if i != 0:
            sd_rec = nusc.get('sample_data', sd_rec['next'])
        ego_pose = nusc.get('ego_pose', sd_rec['token'])
        oxts_data[i] = np.loadtxt(os.path.join(base_dir, ego_pose['token'] + '.txt'))

    return oxts_data

In [4]:
def lat_lon_to_mercator(lat, lon, scale):
    """
    Converts lat/lon coordinates to mercator coordinates using Mercator scale.
    """
    ER = 6378137
    mx = scale * lon * math.pi * ER / 180
    my = scale * ER * math.log(math.tan((90 + lat) * math.pi / 360))
    return mx, my


def lat_to_scale(lat):
    """
    Compute Mercator scale from latitude
    """
    return math.cos(lat * math.pi / 180.0)

In [5]:
def gps_imu_to_pose(gps_imu, scale):
    """
    Compute pose from GPS/IMU data
    """
    t = np.zeros(3)
    t[0], t[1] = lat_lon_to_mercator(gps_imu[0], gps_imu[1], scale)
    t[2] = gps_imu[2]  # altitude
    rx = gps_imu[3]  # roll
    ry = gps_imu[4]  # pitch
    rz = gps_imu[5]  # heading
    Rx = np.array([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]])
    Ry = np.array([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]])
    Rz = np.array([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    T = np.identity(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

In [6]:
def oxts_to_pose(oxts_info):
    """
    Converts a list of oxts measurements into metric poses,
    starting at (0,0,0) meters, OXTS coordinates are defined as
    x = forward, y = right, z = down (see OXTS RT3000 user manual)
    afterwards, pose{i} contains the transformation which takes a
    3D point in the i'th frame and projects it into the oxts
    coordinates of the first frame.
    """
    # Compute scale from first lat value
    scale = lat_to_scale(oxts_info[0, 0])
    Tr_0_inv = None
    poses = np.zeros((len(oxts_info), 12))
    for i, line in enumerate(oxts_info):
        T = gps_imu_to_pose(line, scale)
        # Normalize translation and rotation (start at 0/0/0)
        if Tr_0_inv is None:
            Tr_0_inv = np.linalg.inv(T)

        pose = Tr_0_inv.dot(T)
        poses[i] = pose[:3, :].reshape(12)
    return poses

In [22]:
for scene in nusc.scene:
    scene_name = scene['name']
    scene_token = scene['token']
    oxts_data = load_oxts_data(scene_token, f'{base_dir}{scene_name}/')
    poses = oxts_to_pose(oxts_data)
    np.savetxt(f'{base_dir}poses/{scene_name}.txt', poses)
